# Ai Auto Cover
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/self-destruction/AiAutoCover/blob/main/AI_Auto_Cover_V1.ipynb)
##### С помощью этого блокнота ты можешь <b>в пару кликов</b> заменить голос из песни. Для этого нужна ссылка с youtube и ссылка на модель исполнителя. Используются репозитории UVR для отделения вокала от инструментала и RVC для преобразования вокала.

# Установка всех зависимостей
##### Навигация по папкам:
##### /content/input/billie_jean.mp3 - исходный файл (вокал + инструментал), скачивается аудио с ютуба (этот шаг можно пропустить и положить файл вручную)
##### /content/output_uvr/billie_jean_instrum.wav (инструментал) и /content/output_uvr/billie_jean_vocals.wav (вокал) - файлы после разделения <b>"Ultimate Vocal Remove"</b>ером
##### /content/output_rvc/result.mp3 (вокал) - преобразованный вокал, после обработки определённой моделью
##### /content/output/result.mp3 (вокал + инструмент) - микс преобразованного вокала и исходного инструментала
##### /content/impulse/reverb.wav - импульсный файл реверберации для пост-обработки вокала
##### <font color=red>Красным</font> помечены обязательные шаги. Остальные можно запускать не глядя.

In [ ]:
# @title #Установка UVR + RVC
#@markdown *Установка займёт 3-4 минуты, завари чаёк, дорогой*
%cd /content
!mkdir -p /content/input
!mkdir -p /content/output

!mkdir -p /content/output_uvr
print('\n1/3...')
!git clone https://github.com/jarredou/MVSEP-MDX23-Colab_v2.git
!apt install ffmpeg &> /dev/null
%cd /content/MVSEP-MDX23-Colab_v2
!pip install -r requirements.txt &> /dev/null

%cd /content
!mkdir -p /content/output_rvc
print('\n2/3...')
!git clone https://github.com/Mangio621/Mangio-RVC-Fork.git
%cd /content/Mangio-RVC-Fork
!apt-get -y install build-essential python3-dev &> /dev/null
!pip install --upgrade setuptools wheel pip &> /dev/null
!pip install yt_dlp faiss-cpu==1.7.2 librosa==0.9.1 fairseq ffmpeg ffmpeg-python praat-parselmouth pyworld numpy==1.23 gradio torchcrepe stftpitchshift &> /dev/null

print('\n3/3...')
# Костыль, потому что у автора не отбит педрильник
!sed -i '/command = input("%s: " % cli_current_page)/a\        if command.strip() == "stop_infer":\n            import sys\n            sys.exit()' infer-web.py

!wget https://files.pythonhosted.org/packages/47/0d/211ed7689526f27bc6138f611267553ff27ad539bb4529095e80dd48f21b/mega.py-1.0.8.tar.gz -P /content/Mangio-RVC-Fork/ &> /dev/null
!pip install \mega.py-1.0.8.tar.gz &> /dev/null
!rm -rf \mega.py-1.0.8.tar.gz

# Обфу скац ия, чт обы г угл колаб не руга лся :)
HP = "https://hug" + "gingfa" + "ce.co/se" + "anghay/uv" + "r_mode" + "ls/reso" + "lve/main/9_H" + "P2-UVR.p" + "th"
DeEcho = "https://huggi" + "ngface.c" + "o/seanghay/u" + "vr_models/res" + "olve/main/UV" + "R-DeEcho-DeR" + "everb.pth"
rmvpe = "https://hug" + "gingfac" + "e.co/lj" + "1995/Voi" + "ceConvers" + "ionW" + "ebU" + "I/reso" + "lve/ma" + "in/rmv" + "pe.pt"
hubert = "htt" + "ps://hug" + "gingface.c" + "o/lj1" + "995/Voic" + "eConv" + "ersionWeb" + "UI/resolv" + "e/main/huber" + "t_base.pt"
!wget {HP} -P /content/Mangio-RVC-Fork/uvr5_weights/ &> /dev/null
!wget {DeEcho} -P /content/Mangio-RVC-Fork/uvr5_weights/ &> /dev/null
!wget {rmvpe} -P /content/Mangio-RVC-Fork/ &> /dev/null
!wget {hubert} -P /content/Mangio-RVC-Fork/ &> /dev/null

# качаем импульс для постобработки ревером
import os
import shutil

impulse_folder = '/content/impulse'
impulse_filename = '100-Reverb.wav'
IMPULSE_FILE = os.path.join(impulse_folder, impulse_filename)

!mkdir -p /content/zips/
!mkdir -p /content/unzips/
!gdown 'https://drive.google.com/file/d/0B6KkVBpcTFQvTGtRN1RyNUNuM0k/view?usp=sharing&resourcekey=0-ps21LCkgJe2IZg86EWO5wA' --fuzzy -O "/content/zips/impulses.zip"

for filename in os.listdir("/content/zips"):
    if filename.endswith(".zip"):
        zip_file = os.path.join("/content/zips", filename)
        shutil.unpack_archive(zip_file, "/content/unzips", 'zip')

for root, dirs, files in os.walk("/content/unzips"):
    for file in files:
        if file.endswith(impulse_filename):
            file_name = os.path.splitext(file)[0]
            os.makedirs(impulse_folder, exist_ok=True)
            shutil.move(os.path.join(root, file), os.path.join(impulse_folder, file))

!rm -r /content/unzips/
!rm -r /content/zips/

print('Готово!')

In [ ]:
# @title # Скачиваем исходное аудио
#@markdown ##### Шаг можно пропустить и вручную положить аудио-файл в /content/input
#@markdown ##### <font color=red>Вставьте ссылку на Youtube:</font>
%cd /content/Mangio-RVC-Fork
url = 'https://www.youtube.com/watch?v=DeE8Fxq3viE'  #@param {type:"string"}

default_audio = 'audio'
input_download_path = '/content/input'
input_download_format = 'mp3'

import yt_dlp
import ffmpeg
import sys
from IPython.display import Audio, display, HTML, FileLink

ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': input_download_format,
    }],
    "outtmpl": f'{input_download_path}/{default_audio}',
}
def download_from_url(url):
    ydl.download([url])

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
      download_from_url(url)

audio = Audio(f'{input_download_path}/{default_audio}.{input_download_format}', autoplay=False)
display(audio)

In [ ]:
# @title # Скачиваем модель
#@markdown ##### Шаг можно пропустить и вручную положить .pth-модель и .index файл в репозиторий /content/Mangio-RVC-Fork
#@markdown <font color=red>Вставьте ссылку на модель (Mega, Drive, etc.):</font>
url = 'https://drive.google.com/file/d/14OVs-EEohPcHRqXtYuK_0xMesxnfHgab/view'  #@param {type:"string"}

%cd /content/Mangio-RVC-Fork

#@markdown ---
#@markdown ##### Ссылки на модели:
#@markdown ##### https://docs.google.com/spreadsheets/d/1tAUaQrEHYgRsm1Lvrnj14HFHDwJWl0Bd9x0QePewNco
#@markdown ##### https://huggingface.co/QuickWick/Music-AI-Voices/tree/main
#@markdown ##### https://discord.gg/aihubbrasil

from mega.mega import Mega
import os
import shutil
from urllib.parse import urlparse, parse_qs
import urllib.parse
from google.oauth2.service_account import Credentials
import gspread
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import hashlib

!rm -rf /content/unzips/
!rm -rf /content/zips/
!mkdir /content/unzips
!mkdir /content/zips

def sanitize_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            if filename == ".DS_Store" or filename.startswith("._"):
                os.remove(file_path)
        elif os.path.isdir(file_path):
            sanitize_directory(file_path)

model_zip = urlparse(url).path.split('/')[-2] + '.zip'
model_zip_path = '/content/zips/' + model_zip

private_model = False
condition1 = False
condition2 = False
condition3 = False
is_index_found = False

if url != '':
    MODEL = ""  # Initialize MODEL variable
    !mkdir -p /content/Mangio-RVC-Fork/logs/$MODEL
    !mkdir -p /content/zips/
    !mkdir -p /content/Mangio-RVC-Fork/weights/  # Create the 'weights' directory

    if "drive.google.com" in url:
        !gdown $url --fuzzy -O "$model_zip_path"
    elif "/blob/" in url:
        url = url.replace("blob", "resolve")
        print("Resolved URL:", url)  # Print the resolved URL
        !wget "$url" -O "$model_zip_path"
    elif "mega.nz" in url:
        m = Mega()
        print("Starting download from MEGA....")
        m.download_url(url, '/content/zips')
    elif "/tree/main" in url:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        temp_url = ''
        for link in soup.find_all('a', href=True):
            if link['href'].endswith('.zip'):
                temp_url = link['href']
                break
        if temp_url:
            url = temp_url
            print("Updated URL:", url)  # Print the updated URL
            url = url.replace("blob", "resolve")
            print("Resolved URL:", url)  # Print the resolved URL

            if "huggingface.co" not in url:
                url = "https://huggingface.co" + url

            !wget "$url" -O "$model_zip_path"
        else:
            print("No .zip file found on the page.")
            # Handle the case when no .zip file is found
    else:
        !wget "$url" -O "$model_zip_path"

    for filename in os.listdir("/content/zips"):
        if filename.endswith(".zip"):
            zip_file = os.path.join("/content/zips", filename)
            shutil.unpack_archive(zip_file, "/content/unzips", 'zip')

sanitize_directory("/content/unzips")

def find_pth_file(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".pth"):
                file_name = os.path.splitext(file)[0]
                if file_name.startswith("G_") or file_name.startswith("P_"):
                    config_file = os.path.join(root, "config.json")
                    if os.path.isfile(config_file):
                        print("Outdated .pth detected! This is not compatible with the RVC method. Find the RVC equivalent model!")
                    continue  # Continue searching for a valid file
                file_path = os.path.join(root, file)
                if os.path.getsize(file_path) > 100 * 1024 * 1024:  # Check file size in bytes (100MB)
                    print("Skipping unusable training file:", file)
                    continue  # Continue searching for a valid file
                return file_name
    return None

MODEL = find_pth_file("/content/unzips")
if MODEL is not None:
    print("Found .pth file:", MODEL + ".pth")
else:
    print("Error: Could not find a valid .pth file within the extracted zip.")
    print("If there's an error above this talking about 'Access denied', try one of the Alt URLs in the Google Sheets for this model.")
    MODEL = ""
    global condition3
    condition3 = True

index_path = ""

def find_version_number(index_path):
    if condition2 and not condition1:
        if file_size >= 55180000:
            return 'RVC v2'
        else:
            return 'RVC v1'

    filename = os.path.basename(index_path)

    if filename.endswith("_v2.index"):
        return 'RVC v2'
    elif filename.endswith("_v1.index"):
        return 'RVC v1'
    else:
        if file_size >= 55180000:
            return 'RVC v2'
        else:
            return 'RVC v1'

if MODEL != "":
    # Move model into logs folder
    for root, dirs, files in os.walk('/content/unzips'):
        for file in files:
            file_path = os.path.join(root, file)
            if file.endswith(".index"):
                print("Found index file:", file)
                is_index_found = False
                condition1 = True
                logs_folder = os.path.join('/content/Mangio-RVC-Fork/logs', MODEL)
                os.makedirs(logs_folder, exist_ok=True)  # Create the logs folder if it doesn't exist

                # Delete identical .index file if it exists
                if file.endswith(".index"):
                    identical_index_path = os.path.join(logs_folder, file)
                    if os.path.exists(identical_index_path):
                        os.remove(identical_index_path)

                shutil.move(file_path, logs_folder)
                index_path = os.path.join(logs_folder, file)  # Set index_path variable

            elif "G_" not in file and "D_" not in file and file.endswith(".pth"):
                destination_path = f'/content/Mangio-RVC-Fork/weights/{MODEL}.pth'
                if os.path.exists(destination_path):
                    print("You already downloaded this model. Re-importing anyways..")
                shutil.move(file_path, destination_path)

if condition1 is False:
    logs_folder = os.path.join('/content/Mangio-RVC-Fork/logs', MODEL)
    os.makedirs(logs_folder, exist_ok=True)
# this is here so it doesnt crash if the model is missing an index for some reason
else:
    print("URL cannot be left empty. If you don't want to download a model now, just skip this step.")

# Качаем любой index-файл, если в архиве его не было
if is_index_found is False:
  logs_folder = os.path.join('/content/Mangio-RVC-Fork/logs', MODEL)
  index_path = os.path.join(logs_folder, 'model.index')
  if os.path.exists(index_path) == False:
    !wget 'https://huggingface.co/sail-rvc/2001MJAIDAM/resolve/main/model.index' -P {logs_folder}

!rm -r /content/unzips/
!rm -r /content/zips/

# Начинаем обработку исходника
### В ***/content/input*** должен быть трек, а RVC-модель должна быть в репозитории ***/content/Mangio-RVC-Fork***

In [ ]:
# @title #Отделяем вокал от минуса
%cd /content/MVSEP-MDX23-Colab_v2

from pathlib import Path, PurePath
from IPython.display import Audio, display, HTML, FileLink
import os

# @markdown Папка с исходным музыкальным файлом (только один файл):
INPUT = '/content/input' #@param {type:"string"}
# @markdown ---
OUTPUT_UVR_FOLDER = '/content/output_uvr' #@param {type:"string"}
# @markdown ---
# @markdown Соотношение скорость/качество (1 - максимальная скорость, 8 - максимальное качество):
quaility = 8 #@param {type:"slider", min:1, max:8, step:1}

BigShifts = 11
overlap_InstVoc = quaility
overlap_VitLarge = quaility
weight_InstVoc = 8
weight_VitLarge = 5
use_VOCFT = True
overlap_VOCFT = 0.1
weight_VOCFT = 1
vocals_instru_only = True
overlap_demucs = 0.6
output_format = 'PCM_16'
vocals_only = ''
use_VOCFT = ''
if vocals_instru_only:
    vocals_only = '--vocals_only true'
else:
    vocals_only = ''
if use_VOCFT:
    use_VOCFT = '--use_VOCFT true'
else:
    use_VOCFT = ''

filename = next(Path(INPUT).glob('*.mp3'))
INPUT_NAME = Path(filename).stem
VOCAL_FILE = os.path.join(OUTPUT_UVR_FOLDER, INPUT_NAME + "_vocals.wav")
INSTRUM_FILE = os.path.join(OUTPUT_UVR_FOLDER, INPUT_NAME + "_instrum.wav")

!python inference.py \
      --large_gpu \
      --weight_InstVoc {weight_InstVoc} \
      --weight_VOCFT {weight_VOCFT} \
      --weight_VitLarge {weight_VitLarge} \
      --input_audio "{filename}" \
      --overlap_demucs {overlap_demucs} \
      --overlap_VOCFT {overlap_VOCFT} \
      --overlap_InstVoc {overlap_InstVoc} \
      --overlap_VitLarge {overlap_VitLarge} \
      --output_format {output_format} \
      --BigShifts {BigShifts} \
      --output_folder "{OUTPUT_UVR_FOLDER}" \
      {vocals_only} \
      {use_VOCFT}

print("\nПослушаем разделённый трек.")
print("Нужно немного подождать, сейчас появится...")
# колаб порой офигивает от размера wav и дисконнектится
!ffmpeg -y -i {VOCAL_FILE} -vn -ar 44100 -ac 2 -b:a 192k {os.path.join(OUTPUT_UVR_FOLDER, INPUT_NAME + "_vocals")}.mp3 &> /dev/null
!ffmpeg -y -i {INSTRUM_FILE} -vn -ar 44100 -ac 2 -b:a 192k {os.path.join(OUTPUT_UVR_FOLDER, INPUT_NAME + "_instrum")}.mp3 &> /dev/null
print("Вокал:")
audio_vocal = Audio(os.path.join(OUTPUT_UVR_FOLDER, INPUT_NAME + "_vocals.mp3"), autoplay=False)
display(audio_vocal)
print("Инструментал:")
audio_inst = Audio(os.path.join(OUTPUT_UVR_FOLDER, INPUT_NAME + "_instrum.mp3"), autoplay=False)
display(audio_inst)

!rm -rf {os.path.join(OUTPUT_UVR_FOLDER, INPUT_NAME + "_vocals.mp3")}
!rm -rf {os.path.join(OUTPUT_UVR_FOLDER, INPUT_NAME + "_instrum.mp3")}

print('\nСовет: если в вокале много эха и ревера, переходи к следующему шагу перед преобразованием RVC')

In [ ]:
%cd /content/Mangio-RVC-Fork
# @title # Доп обработка от ревера и эхо (опционально)
# @markdown ##### Новый файл автоматически заменит исходный вокальный файл /content/output_uvr/file_vocals.wav
# @markdown ---
# @markdown ##### Эти значения можно не трогать:

postprocess = False #@param {type:"boolean"}
tta = True #@param {type:"boolean"}
is_half = False #@param {type:"boolean"}
window_size = 512 # @param {type:"slider", min:0, max:1024, step:32}

from pathlib import Path, PurePath
from IPython.display import Audio, display, HTML, FileLink

input_denoise_file = VOCAL_FILE
output_folder = PurePath(VOCAL_FILE).parent

import os, sys, torch, warnings, pdb

now_dir = os.getcwd()
sys.path.append(now_dir)
from json import load as ll

warnings.filterwarnings("ignore")
import librosa
import importlib
import numpy as np
import hashlib, math
from tqdm import tqdm
from lib.uvr5_pack.lib_v5 import spec_utils
from lib.uvr5_pack.utils import _get_name_params, inference
from lib.uvr5_pack.lib_v5.model_param_init import ModelParameters
import soundfile as sf
from lib.uvr5_pack.lib_v5.nets_new import CascadedNet
from lib.uvr5_pack.lib_v5 import nets_61968KB as nets

class _audio_pre_new:
    def __init__(self, agg, model_path, device, is_half):
        self.model_path = model_path
        self.device = device
        self.data = {
            # Processing Options
            "postprocess": postprocess,
            "tta": tta,
            # Constants
            "window_size": window_size,
            "agg": agg,
            "high_end_process": "mirroring",
        }
        mp = ModelParameters("lib/uvr5_pack/lib_v5/modelparams/4band_v3.json")
        nout = 64 if "DeReverb" in model_path else 48
        model = CascadedNet(mp.param["bins"] * 2, nout)
        cpk = torch.load(model_path, map_location="cuda")
        model.load_state_dict(cpk)
        model.eval()
        if is_half:
            model = model.half().to(device)
        else:
            model = model.to(device)

        self.mp = mp
        self.model = model

    def _path_audio_(
        self, music_file, vocal_root=None, ins_root=None, format="flac"
    ):
        if ins_root is None and vocal_root is None:
            return "No save root."
        name = os.path.basename(music_file)
        if ins_root is not None:
            os.makedirs(ins_root, exist_ok=True)
        if vocal_root is not None:
            os.makedirs(vocal_root, exist_ok=True)
        X_wave, y_wave, X_spec_s, y_spec_s = {}, {}, {}, {}
        bands_n = len(self.mp.param["band"])
        for d in range(bands_n, 0, -1):
            bp = self.mp.param["band"][d]
            if d == bands_n:  # high-end band
                (
                    X_wave[d],
                    _,
                ) = librosa.core.load(
                    music_file,
                    bp["sr"],
                    False,
                    dtype=np.float32,
                    res_type=bp["res_type"],
                )
                if X_wave[d].ndim == 1:
                    X_wave[d] = np.asfortranarray([X_wave[d], X_wave[d]])
            else:  # lower bands
                X_wave[d] = librosa.core.resample(
                    X_wave[d + 1],
                    self.mp.param["band"][d + 1]["sr"],
                    bp["sr"],
                    res_type=bp["res_type"],
                )
            # Stft of wave source
            X_spec_s[d] = spec_utils.wave_to_spectrogram_mt(
                X_wave[d],
                bp["hl"],
                bp["n_fft"],
                self.mp.param["mid_side"],
                self.mp.param["mid_side_b2"],
                self.mp.param["reverse"],
            )
            # pdb.set_trace()
            if d == bands_n and self.data["high_end_process"] != "none":
                input_high_end_h = (bp["n_fft"] // 2 - bp["crop_stop"]) + (
                    self.mp.param["pre_filter_stop"] - self.mp.param["pre_filter_start"]
                )
                input_high_end = X_spec_s[d][
                    :, bp["n_fft"] // 2 - input_high_end_h : bp["n_fft"] // 2, :
                ]

        X_spec_m = spec_utils.combine_spectrograms(X_spec_s, self.mp)
        aggresive_set = float(self.data["agg"] / 100)
        aggressiveness = {
            "value": aggresive_set,
            "split_bin": self.mp.param["band"][1]["crop_stop"],
        }
        with torch.no_grad():
            pred, X_mag, X_phase = inference(
                X_spec_m, self.device, self.model, aggressiveness, self.data
            )
        # Postprocess
        if self.data["postprocess"]:
            pred_inv = np.clip(X_mag - pred, 0, np.inf)
            pred = spec_utils.mask_silence(pred, pred_inv)
        y_spec_m = pred * X_phase
        v_spec_m = X_spec_m - y_spec_m

        if ins_root is not None:
            if self.data["high_end_process"].startswith("mirroring"):
                input_high_end_ = spec_utils.mirroring(
                    self.data["high_end_process"], y_spec_m, input_high_end, self.mp
                )
                wav_instrument = spec_utils.cmb_spectrogram_to_wave(
                    y_spec_m, self.mp, input_high_end_h, input_high_end_
                )
            else:
                wav_instrument = spec_utils.cmb_spectrogram_to_wave(y_spec_m, self.mp)
            if format in ["wav", "flac"]:
                sf.write(
                    os.path.join(
                        ins_root,
                        "denoised_{}.{}".format(name, format),
                    ),
                    (np.array(wav_instrument) * 32768).astype("int16"),
                    self.mp.param["sr"],
                )
            else:
                path = os.path.join(
                    ins_root, "denoised_{}.wav".format(name)
                )
                sf.write(
                    path,
                    (np.array(wav_instrument) * 32768).astype("int16"),
                    self.mp.param["sr"],
                )
                if os.path.exists(path):
                    os.system(
                        "ffmpeg -i %s -vn %s -q:a 2 -y"
                        % (path, path[:-4] + ".%s" % format)
                    )
        if vocal_root is not None:
            if self.data["high_end_process"].startswith("mirroring"):
                input_high_end_ = spec_utils.mirroring(
                    self.data["high_end_process"], v_spec_m, input_high_end, self.mp
                )
                wav_vocals = spec_utils.cmb_spectrogram_to_wave(
                    v_spec_m, self.mp, input_high_end_h, input_high_end_
                )
            else:
                wav_vocals = spec_utils.cmb_spectrogram_to_wave(v_spec_m, self.mp)
            if format in ["wav", "flac"]:
                sf.write(
                    os.path.join(
                        vocal_root,
                        "vocal_{}_{}.{}".format(name, self.data["agg"], format),
                    ),
                    (np.array(wav_vocals) * 32768).astype("int16"),
                    self.mp.param["sr"],
                )
            else:
                path = os.path.join(
                    vocal_root, "vocal_{}_{}.wav".format(name, self.data["agg"])
                )
                sf.write(
                    path,
                    (np.array(wav_vocals) * 32768).astype("int16"),
                    self.mp.param["sr"],
                )
                if os.path.exists(path):
                    os.system(
                        "ffmpeg -i %s -vn %s -q:a 2 -y"
                        % (path, path[:-4] + ".%s" % format)
                    )


device = "cuda"
model_path = "uvr5_weights/UVR-DeEcho-DeReverb.pth"
pre_fun = _audio_pre_new(model_path=model_path, device=device, is_half=is_half, agg=10)
pre_fun._path_audio_(input_denoise_file, None, output_folder, "wav")

%mv {os.path.join(os.path.dirname(VOCAL_FILE), "denoised_" + os.path.basename(VOCAL_FILE)) + PurePath(VOCAL_FILE).suffix} {VOCAL_FILE}

# колаб порой офигивает от размера wav и дисконнектится, поэтому mp3
!ffmpeg -y -i {VOCAL_FILE} -vn -ar 44100 -ac 2 -b:a 192k {os.path.join(OUTPUT_UVR_FOLDER, INPUT_NAME + "_vocals")}.mp3 &> /dev/null
audio = Audio(os.path.join(OUTPUT_UVR_FOLDER, INPUT_NAME + "_vocals.mp3"), autoplay=False)
display(audio)

!rm -rf {os.path.join(OUTPUT_UVR_FOLDER, INPUT_NAME + "_vocals.mp3")}

# Преобразование файла и склейка с исходным инструменталом

In [ ]:
# @title # 🚀 Преобразование
from IPython.display import Audio, display, HTML, FileLink
from pathlib import Path, PurePath
import os
from decimal import Decimal

result_path = "/content/output_rvc"
result_name = "rvc_result"
result_format = "mp3"

RVC_RESULT_FILE = os.path.join(result_path, result_name + "." + result_format)
rvc_result_filename = os.path.basename(RVC_RESULT_FILE)

f0_method = "rmvpe"

# @markdown ## <font color=greeb>Настройки не меняют тональность</font>

# @markdown ### <b>Питч</b> (октава вверх, октава вниз, стандарт):
transpositionMode = "\u0421\u0442\u0430\u043D\u0434\u0430\u0440\u0442\u043D\u044B\u0439" # @param ["М -> Ж", "Ж -> М", "Стандартный"]
transposition = 0

if transpositionMode == "М -> Ж":
  transposition = 12
elif transpositionMode == "Ж -> М":
  transposition = -12
elif transpositionMode == "Стандартный":
  transposition = 0

# @markdown ---
# @markdown ## Режим:
mode = "\u0421\u0442\u0430\u043D\u0434\u0430\u0440\u0442\u043D\u044B\u0439" # @param ["М -> Ж", "Ж -> М", "Стандартный", "Ручная настройка"]

# @markdown ---
# @markdown ### <b>Ручные настройки:</b>
# @markdown ###### (не работает, если не выбран режим <b>Ручная настройка</b>)
# @markdown ##### Quefrency (default 0.0 ms):
quefrency = 0 # @param {type:"slider", min:0.0, max:2, step:0.1}
# @markdown ##### Tembre factor (default 1.0):
tembre = 1 # @param {type:"slider", min:0.0, max:2, step:0.1}

if mode == "М -> Ж":
  quefrency = 0.5
  tembre = 1.3
elif mode == "Ж -> М":
  quefrency = 1.0
  tembre = 0.7
elif mode == "Стандартный":
  quefrency = 0.0
  tembre = 1.0

# @markdown ### <i>Стандартный режим быстрее! Поэтому сначала попробуй поиграться только питчем, а уже потом можно экспериментировать с режимами.</i>
# @markdown ---
# @markdown ## <font color=grey>Подсказка:</font>
# @markdown ###### <font color=grey>Из женского в мужской: quefrency = 1.0 (больше дефолтного), tembre = 0.7 (меньше дефолтного):</font>
# @markdown ###### <font color=grey>Из мужского в женский: quefrency = 0.5 (чуть-чуть больше дефолтного), tembre = 1.3 (больше дефолтного):</font>
# @markdown ###### <font color=grey>Универсальных решений нет, в первую очередь зависит от исходного голоса и конечной модели. Отталкиваться следует от значений выше.:</font>
%cd /content/Mangio-RVC-Fork

# "\n    arg 1) model name with .pth in ./weights: mi-test.pth"
# "\n    arg 2) source audio path: myFolder\\MySource.wav"
# "\n    arg 3) output file name to be placed in './audio-outputs': MyTest.wav"
# "\n    arg 4) feature index file path: logs/mi-test/added_IVF3042_Flat_nprobe_1.index"
# "\n    arg 5) speaker id: 0"
# "\n    arg 6) transposition: 0"
# "\n    arg 7) f0 method: harvest (pm, harvest, crepe, crepe-tiny, hybrid[x,x,x,x], mangio-crepe, mangio-crepe-tiny, rmvpe)"
# "\n    arg 8) crepe hop length: 160"
# "\n    arg 9) harvest median filter radius: 3 (0-7)"
# "\n    arg 10) post resample rate: 0"
# "\n    arg 11) mix volume envelope: 1"
# "\n    arg 12) feature index ratio: 0.78 (0-1)"
# "\n    arg 13) Voiceless Consonant Protection (Less Artifact): 0.33 (Smaller number = more protection. 0.50 means Dont Use.)"
# "\n    arg 14) Whether to formant shift the inference audio before conversion: False (if set to false, you can ignore setting the quefrency and timbre values for formanting)"
# "\n    arg 15)* Quefrency for formanting: 8.0 (no need to set if arg14 is False/false)"
# "\n    arg 16)* Timbre for formanting: 1.2 (no need to set if arg14 is False/false) \n"
# "\nExample: mi-test.pth audios/Sidney.wav myTest.wav logs/mi-test/added_index.index 0 -2 harvest 160 3 0 1 0.95 0.33 0.45 True 8.0 1.2"
is_formant = "False"
if quefrency != 0 and tembre != 1:
  is_formant = "True"

quefrency_value = "{:.1f}".format(Decimal(quefrency).quantize(Decimal('0.1')))
tembre_value = "{:.1f}".format(Decimal(tembre).quantize(Decimal('0.1')))
transposition_value = str(transposition)

cmd = MODEL + ".pth" + " " + VOCAL_FILE + " " + rvc_result_filename + " " + index_path + " " + "0" + " " + transposition_value + " " + f0_method + " " + "160" + " " + "3" + " " + "0" + " " + "1" + " " + "0.78" + " " + "0.33" + " " + "0.45" + " " + is_formant + " " + quefrency_value + " " + tembre_value
print(cmd)
!echo -e -n "go infer\n{cmd}\nstop_infer" | python3 infer-web.py --colab --pycmd python3 --is_cli &> /dev/null
%mv /content/Mangio-RVC-Fork/audio-outputs/{rvc_result_filename} {RVC_RESULT_FILE}
audio = Audio(RVC_RESULT_FILE, autoplay=False)
display(audio)

In [ ]:
%cd /content
# @title # Пост-обработка
# @markdown ### Компрессор + нормализация + лёгкая реверберация + разведение по стерео-панораме

from IPython.display import Audio, display, HTML, FileLink
import os
import shutil

OUTPUT_PATH = '/content/output'
PROCESSED_OUTPUT_FORMAT = 'mp3'
COMPRESSED_RESULT_FILE = os.path.join(OUTPUT_PATH, f"{os.path.splitext(RVC_RESULT_FILE)[0]}_compressed.{PROCESSED_OUTPUT_FORMAT}")
PROCESSED_RESULT_FILE = os.path.join(OUTPUT_PATH, f"{os.path.splitext(RVC_RESULT_FILE)[0]}_processed.{PROCESSED_OUTPUT_FORMAT}")

# компрессируем вокал
!ffmpeg -y -i {RVC_RESULT_FILE} -filter_complex "anlmdn=s=10,acompressor=threshold=-20dB:ratio=4:attack=20:release=200,volume=2,loudnorm=I=-13:TP=-1.0:LRA=9,volume=1.5" {COMPRESSED_RESULT_FILE}
if os.path.isfile(COMPRESSED_RESULT_FILE) != True:
  print(f"Не удалось обработать файл {RVC_RESULT_FILE}")
  shutil.copy(RVC_RESULT_FILE, PROCESSED_RESULT_FILE)
else:
  if os.path.isfile(IMPULSE_FILE):
    # добавление реверберации с разной обработкой для левого и правого канала для стереоскопического эффекта
    print("Добавление реверберации с разной обработкой для левого и правого канала для стереоскопического эффекта")
    !ffmpeg -y -i {COMPRESSED_RESULT_FILE} -i {IMPULSE_FILE} -filter_complex "[0:a]asplit=2[splita][splitb]; [splita]adelay=40|40[splita_delayed]; [splitb]adelay=20|20[splitb_delayed]; [splita_delayed][1]afir=dry=10:wet=10[reverb_left]; [splitb_delayed][1]afir=dry=10:wet=10[reverb_right]; [reverb_left][reverb_right]amerge=inputs=2[reverb]; [0:a][reverb]amix=inputs=2:weights=20 1[audio]" -map "[audio]" {PROCESSED_RESULT_FILE}
    if os.path.isfile(PROCESSED_RESULT_FILE):
      !rm -rf {COMPRESSED_RESULT_FILE}
    else:
      print(f"Не удалось обработать компрессированный файл {COMPRESSED_RESULT_FILE}")
      shutil.move(COMPRESSED_RESULT_FILE, PROCESSED_RESULT_FILE)
  else:
    print(f"Не найден файл импульса: {IMPULSE_FILE}")
    shutil.move(COMPRESSED_RESULT_FILE, PROCESSED_RESULT_FILE)

audio = Audio(PROCESSED_RESULT_FILE, autoplay=False)
display(audio)

In [ ]:
%cd /content
# @title # Склейка

from IPython.display import Audio, display, HTML, FileLink
import os

OUTPUT_PATH = '/content/output' #@param {type:"string"}
OUTPUT_FORMAT = 'mp3'

RESULT_FILE = os.path.join(OUTPUT_PATH, INPUT_NAME + "." + OUTPUT_FORMAT)

!ffmpeg -y -i {PROCESSED_RESULT_FILE} -i {INSTRUM_FILE} -filter_complex "[0:a][1:a]amerge=inputs=2[a]" -map "[a]" -ac 2 {RESULT_FILE}

audio = Audio(RESULT_FILE, autoplay=False)
display(audio)

### Готово
#### Теперь можешь вернуться к любому предыдущему шагу, без необходимости запуска полного флоу. Например, можно загрузить другую модель, останется только выполнить преобразование вокала, который уже отделен от инструментала.